In [8]:
import requests
import pandas as pd
import configparser
import time
from io import StringIO

config = configparser.ConfigParser()
config.read('../conf/config.ini')

# Access values from the configuration file
API_KEY = config.get('Alpha_Vantage_API', 'api_key')
symbol = 'TSLA'
interval = '1min'

all_data = []

months = [
    "2023-05", "2023-06", "2023-07", "2023-08", "2023-09", "2023-10", "2023-11", "2023-12",
    "2024-01", "2024-02", "2024-03", "2024-04",
    "2024-05", "2024-06", "2024-07",  "2024-08", "2024-09", "2024-10","2024-11","2024-12",
    "2025-01","2025-02", "2025-03","2025-04"
]

def concat_month(months):
    final_df = pd.DataFrame()
    for month in months:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&month={month}&outputsize=full&apikey={API_KEY}'
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            print(data)
            if 'Time Series' not in data:
                print(f"⚠️ No data found for {month}.")
                continue
            df = pd.DataFrame.from_dict(data[f'Time Series ({interval})'], orient='index')
            df.columns = ['open', 'high', 'low', 'close', 'volume']
            df.index = pd.to_datetime(df.index)
            df = df.sort_index()
            final_df = pd.concat([final_df, df])
            
            print(f"✅ Retrieved {month}, {len(df)} rows.")
        else:
            print(f"❌ Failed to retrieve {month}, status: {response.status_code}")
    
        time.sleep(15)

    return final_df



# if all_data:
#     final_df = pd.concat(all_data)
#     final_df.to_csv("tsla_intraday_202407_202505.csv", index=False)
# else:
#     print("⚠️ No data downloaded. Check your API key or response messages.")

In [ ]:
final_df = concat_month(months)

In [ ]:
final_df

In [ ]:
final_df.to_csv("../data/stocks/tsla_intraday_202407_202505.csv", index=True)

In [27]:
import requests
import pandas as pd
import configparser
import time
from io import StringIO
import pytz

df_price = pd.read_csv('../data/stocks/tsla_intraday_202305_202504-1m.csv', sep=",", skiprows=1)
df_price.columns = ["Datetime", "Open", "High", "Low", "Close", "Volume"]
df_price.columns
# Ensure Date is datetime 
df_price["Datetime"] = pd.to_datetime(df_price["Datetime"])
eastern = pytz.timezone("US/Eastern")
eastern = pytz.timezone("US/Eastern")
df_price["Datetime"] = pd.to_datetime(df_price["Datetime"]).dt.tz_localize(eastern)
df_price["Date"] = pd.to_datetime(df_price["Datetime"]).dt.date
df_price["Date"] = pd.to_datetime(df_price["Date"])

# Sort by datetime
df_price = df_price.sort_values("Datetime").copy()

In [29]:
df_exchange = pd.read_csv('../data/stocks/usd_eur_230501-250501.csv', sep=",", skiprows=1)
df_exchange.columns = ["Date", "Open", "High", "Low", "Close"]
df_exchange["Date"] = pd.to_datetime(df_exchange["Date"])
df_price = df_price.merge(df_exchange, how='left', on = "Date")
df_price['Close_EUR'] = df_price['Close_x'] * df_price['Close_y']  #  conversion rate
df_price = df_price.drop(columns=['Open_y', 'High_y', 'Low_y', 'Close_y'])

In [4]:
df_exchange

,Date,Open_1,High_1,Low_1,Close_1
0,2025-04-30,0.8780,0.8832,0.8770,0.8823
1,2025-04-29,0.8753,0.8791,0.8752,0.8778
2,2025-04-28,0.8797,0.8822,0.8750,0.8752
3,2025-04-25,0.8778,0.8833,0.8777,0.8796
4,2025-04-24,0.8827,0.8830,0.8771,0.8779
...,...,...,...,...,...
518,2023-05-05,0.9078,0.9112,0.9049,0.9072
519,2023-05-04,0.9032,0.9098,0.9013,0.9075
520,2023-05-03,0.9087,0.9087,0.9016,0.9036
521,2023-05-02,0.9109,0.9135,0.9082,0.9087


In [31]:
df_price = df_price.drop(columns=['Open_x', 'High_x', 'Low_x', 'Open', 'High', 'Low', 'Close'])
df_price

,Datetime,Close_x,Volume,Date,Close_EUR
0,2023-05-01 04:01:00-04:00,164.6000,2880,2023-05-01,149.917680
1,2023-05-01 04:03:00-04:00,164.2800,968,2023-05-01,149.626224
2,2023-05-01 04:04:00-04:00,164.1500,100,2023-05-01,149.507820
3,2023-05-01 04:05:00-04:00,163.7000,2500,2023-05-01,149.097960
4,2023-05-01 04:06:00-04:00,163.4000,1817,2023-05-01,148.824720
...,...,...,...,...,...
479785,2025-04-30 19:55:00-04:00,282.6697,5390,2025-04-30,249.399476
479786,2025-04-30 19:56:00-04:00,282.5000,4623,2025-04-30,249.249750
479787,2025-04-30 19:57:00-04:00,282.5100,8724,2025-04-30,249.258573
479788,2025-04-30 19:58:00-04:00,282.5896,5464,2025-04-30,249.328804


In [23]:
import requests

keyword = "EUR"

url = "https://www.alphavantage.co/query"
params = {
    "function": "SYMBOL_SEARCH",
    "keywords": keyword,
    "apikey": API_KEY
}

response = requests.get(url, params=params)
data = response.json()

# Display matching symbols and their details
matches = data.get("bestMatches", [])
print(f"Found {len(matches)} matches for '{keyword}':")
for match in matches:
    print(f"- {match['1. symbol']} | {match['2. name']} | Region: {match['4. region']} | Type: {match['3. type']}")


Found 0 matches for 'EUR':


In [ ]:
symbol = 'TL0.FRK'
all_data = []
TLGER_df = concat_month(months)

In [22]:
import yfinance as yf

# Get Tesla stock price history
from datetime import datetime, timedelta

start_date = datetime(2023, 5, 1)
end_date = datetime(2023, 12, 31)
dfs = []

while start_date < end_date:
    next_date = start_date + timedelta(days=5)
    df = yf.download("TL0.DE", start=start_date.strftime("%Y-%m-%d"), end=next_date.strftime("%Y-%m-%d"), interval="1m")
    dfs.append(df)
    start_date = next_date
    time.sleep(2)

tsl_de = pd.concat(dfs)
#tsla.to_csv('/Users/phuongkhanh/Documents/Master /coin seminar/COIN_Repo/data/stocks/tsla_stock_230501-250501.csv')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TL0.DE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TL0.DE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time
from twelvedata import TDClient
import configparser

# Load API key
TD_API_KEY = config.get('Twelve_Data_API', 'api_key')

# Initialize Twelve Data client
td = TDClient(apikey=TD_API_KEY)

# Load event dates
event_dates = pd.read_csv('../data/stocks/tesla_event_dates.csv')
event_dates = event_dates['event_date'].tolist()

rate_df = []

for date_str in event_dates:
    date = datetime.strptime(date_str, "%Y-%m-%d")
    start_date_str = (date + timedelta(days=1)).strftime('%Y-%m-%d')
    end_date_str = (date + timedelta(days=2)).strftime('%Y-%m-%d')
    print(f"📅 Fetching: {start_date_str} to {end_date_str}")

    success = False
    while not success:
        try:
            ts = td.time_series(
                symbol="EUR/USD",
                interval="1min",
                start_date=start_date_str,
                end_date=end_date_str,
                outputsize=5000
            )
            df = ts.as_pandas()

            if df is None or df.empty:
                print(f"⚠️ No data for {start_date_str}")
                break

            df["event_date"] = date_str
            rate_df.append(df)
            success = True

        except Exception as e:
            if "run out" in str(e) or "credits" in str(e).lower():
                print("⏳ Rate limit hit. Waiting for 60 seconds...")
                time.sleep(60)
            else:
                print(f"❌ Error on {start_date_str}: {e}")
                break

# Combine all dataframes
if rate_df:
    exchange_rate = pd.concat(rate_df).reset_index()
    print("✅ All data combined.")
else:
    print("⚠️ No valid data retrieved.")

📅 Fetching: 2023-05-12 to 2023-05-13
📅 Fetching: 2023-06-04 to 2023-06-05
❌ Error on 2023-06-04: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-06-09 to 2023-06-10
📅 Fetching: 2023-07-16 to 2023-07-17
❌ Error on 2023-07-16: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-09-18 to 2023-09-19
📅 Fetching: 2023-10-04 to 2023-10-05
📅 Fetching: 2023-11-23 to 2023-11-24
📅 Fetching: 2023-12-06 to 2023-12-07
📅 Fetching: 2024-01-03 to 2024-01-04
⏳ Rate limit hit. Waiting for 60 seconds...
📅 Fetching: 2024-01-09 to 2024-01-10
📅 Fetching: 2024-01-12 to 2024-01-13
📅 Fetching: 2024-02-03 to 2024-02-04
📅 Fetching: 2024-02-12 to 2024-02-13
📅 Fetching: 2024-02-17 to 2024-02-18
📅 Fetching: 2024-02-28 to 2024-02-29
📅 Fetching: 2024-03-05 to 2024-03-06
📅 Fetching: 2024-03-08 to 2024-03-09
⏳ Rate limit hit. Waiting for 60 seconds...
📅 Fetching: 2024-03-13 to 2024-03-14
📅 Fetching: 2024-03-14 to 2024-0

In [ ]:
exchange_rate

,datetime,open,high,low,close,event_date
0,2023-05-12 14:00:00,1.09220,1.09230,1.09220,1.09230,2023-05-11
1,2023-05-12 13:59:00,1.09220,1.09230,1.09220,1.09230,2023-05-11
2,2023-05-12 13:58:00,1.09220,1.09230,1.09220,1.09230,2023-05-11
3,2023-05-12 13:57:00,1.09210,1.09220,1.09210,1.09220,2023-05-11
4,2023-05-12 13:56:00,1.09210,1.09215,1.09210,1.09215,2023-05-11
...,...,...,...,...,...,...
120621,2025-03-31 13:01:00,1.08312,1.08324,1.08305,1.08313,2025-03-31
120622,2025-03-31 13:00:00,1.08327,1.08333,1.08310,1.08314,2025-03-31
120623,2025-03-31 12:59:00,1.08330,1.08336,1.08318,1.08326,2025-03-31
120624,2025-03-31 12:58:00,1.08322,1.08345,1.08315,1.08335,2025-03-31


In [ ]:
exchange_rate.to_csv('../data/stocks/EURUSD_event_data.csv', index=True)

In [6]:
from datetime import datetime, timedelta

# Create date list 
# Start and end dates
start_date = datetime(2023, 5, 1)
end_date = datetime(2025, 5, 1)

# Generate list of dates
date_list = []
current = start_date
while current <= end_date:
    date_list.append(current)
    current += timedelta(days=1)

# Example: print first 5 dates
print(date_list[:5])

[datetime.datetime(2023, 5, 1, 0, 0), datetime.datetime(2023, 5, 2, 0, 0), datetime.datetime(2023, 5, 3, 0, 0), datetime.datetime(2023, 5, 4, 0, 0), datetime.datetime(2023, 5, 5, 0, 0)]


In [12]:
from datetime import datetime, timedelta
import pandas as pd
import time
from twelvedata import TDClient
import configparser

# Load API key
TD_API_KEY = config.get('Twelve_Data_API', 'api_key')

# Initialize Twelve Data client
td = TDClient(apikey=TD_API_KEY)

# Load event dates
event_dates = pd.read_csv('../data/stocks/tesla_event_dates.csv')
event_dates = event_dates['event_date'].tolist()

GER_df = []

for date in date_list:
    #date = datetime.strptime(date_str, "%Y-%m-%d")
    start_date_str = (date + timedelta(days=1)).strftime('%Y-%m-%d')
    end_date_str = (date + timedelta(days=2)).strftime('%Y-%m-%d')
    print(f"📅 Fetching: {start_date_str} to {end_date_str}")

    success = False
    while not success:
        try:
            ts = td.time_series(
                symbol="TL0",
                interval="1min",
                start_date=start_date_str,
                end_date=end_date_str,
                outputsize=5000,
                timezone="Europe/Zurich"  # Adjust timezone if necessary
            )
            df = ts.as_pandas()

            if df is None or df.empty:
                print(f"⚠️ No data for {start_date_str}")
                break

            GER_df.append(df)
            success = True

        except Exception as e:
            if "run out" in str(e) or "credits" in str(e).lower():
                print("⏳ Rate limit hit. Waiting for 60 seconds...")
                time.sleep(60)
            else:
                print(f"❌ Error on {start_date_str}: {e}")
                break

# Combine all dataframes
if GER_df:
    TSL_GER = pd.concat(GER_df).reset_index()
    print("✅ All data combined.")
else:
    print("⚠️ No valid data retrieved.")

📅 Fetching: 2023-05-02 to 2023-05-03
📅 Fetching: 2023-05-03 to 2023-05-04
📅 Fetching: 2023-05-04 to 2023-05-05
📅 Fetching: 2023-05-05 to 2023-05-06
📅 Fetching: 2023-05-06 to 2023-05-07
❌ Error on 2023-05-06: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-05-07 to 2023-05-08
❌ Error on 2023-05-07: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-05-08 to 2023-05-09
📅 Fetching: 2023-05-09 to 2023-05-10
📅 Fetching: 2023-05-10 to 2023-05-11
📅 Fetching: 2023-05-11 to 2023-05-12
📅 Fetching: 2023-05-12 to 2023-05-13
📅 Fetching: 2023-05-13 to 2023-05-14
❌ Error on 2023-05-13: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-05-14 to 2023-05-15
❌ Error on 2023-05-14: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2023-05-15 to 2023-05-16
📅 Fetching: 2023-05-16 to 2023-05-17
📅 Fetching: 2023-

/Users/phuongkhanh/Documents/Master /coin seminar/COIN_Repo/.venv/lib/python3.10/site-packages/twelvedata/mixins.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)


📅 Fetching: 2024-10-01 to 2024-10-02
📅 Fetching: 2024-10-02 to 2024-10-03
📅 Fetching: 2024-10-03 to 2024-10-04
📅 Fetching: 2024-10-04 to 2024-10-05
📅 Fetching: 2024-10-05 to 2024-10-06
❌ Error on 2024-10-05: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2024-10-06 to 2024-10-07
❌ Error on 2024-10-06: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2024-10-07 to 2024-10-08
📅 Fetching: 2024-10-08 to 2024-10-09
📅 Fetching: 2024-10-09 to 2024-10-10
📅 Fetching: 2024-10-10 to 2024-10-11
📅 Fetching: 2024-10-11 to 2024-10-12
📅 Fetching: 2024-10-12 to 2024-10-13
❌ Error on 2024-10-12: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2024-10-13 to 2024-10-14
❌ Error on 2024-10-13: No data is available on the specified dates. Try setting different start/end dates.
📅 Fetching: 2024-10-14 to 2024-10-15
📅 Fetching: 2024-10-15 to 2024-10-16
📅 Fetching: 2024-

In [13]:
TSL_GER.to_csv('../data/stocks/TSL0_intraday_230501_250501.csv', index=True)